In [5]:
import torch
from torch import nn
import numpy as np
import pandas
from torch.utils.data import Dataset, DataLoader
import time

In [92]:
class Network(nn.Module):
        def __init__(self):
                super().__init__()

                # Layer sizes
                self.input = nn.Linear(6, 100)
                self.first_hidden = nn.Linear(100, 250)
                self.second_hidden = nn.Linear(250, 250)
                self.third_hidden = nn.Linear(250, 100)
                self.fourth_hidden = nn.Linear(100, 50)
                self.output = nn.Linear(50, 1)

                # Activation functions
                self.relu = nn.ReLU()
                self.sigmoid = nn.Sigmoid()

        def forward(self, x:[])->[]:
                x = self.input(x)
                x = self.relu(x)
                x = self.first_hidden(x)
                x = self.relu(x)
                x = self.second_hidden(x)
                x = self.relu(x)
                x = self.third_hidden(x)
                x = self.relu(x)
                x = self.fourth_hidden(x)
                x = self.relu(x)
                x = self.output(x)

                return x

In [30]:
pandas.read_csv('DATA_FILES/dataset_with_row_id.csv')

,Unnamed: 0,Fill Factor,Pitch,Duty Cycle,Theta,Lambda,Mode,Transmission
0,0,0.2,5.000000e-07,0.4,5,0.000002,1,0.000097
1,1,0.2,5.000000e-07,0.4,5,0.000002,1,0.000098
2,2,0.2,5.000000e-07,0.4,5,0.000002,1,0.000100
3,3,0.2,5.000000e-07,0.4,5,0.000002,1,0.000102
4,4,0.2,5.000000e-07,0.4,5,0.000002,1,0.000104
...,...,...,...,...,...,...,...,...
518395,518395,0.6,9.000000e-07,0.8,20,0.000001,0,-0.012646
518396,518396,0.6,9.000000e-07,0.8,20,0.000001,0,-0.012310
518397,518397,0.6,9.000000e-07,0.8,20,0.000001,0,-0.012144
518398,518398,0.6,9.000000e-07,0.8,20,0.000001,0,-0.011917


In [93]:
GratingCouplerNet = torch.load('GratingCouplerNetModel')
GratingCouplerNet.eval()

Network(
  (input): Linear(in_features=6, out_features=100, bias=True)
  (first_hidden): Linear(in_features=100, out_features=250, bias=True)
  (second_hidden): Linear(in_features=250, out_features=250, bias=True)
  (third_hidden): Linear(in_features=250, out_features=100, bias=True)
  (fourth_hidden): Linear(in_features=100, out_features=50, bias=True)
  (output): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [34]:
training_set = pandas.read_csv('DATA_FILES/training_set_normalized_features_and_labels.csv')
testing_set = pandas.read_csv('DATA_FILES/testing_set_normalized_features_and_labels.csv')

In [48]:
testing_set

,Fill Factor,Pitch,Duty Cycle,Theta,Lambda,Mode,Transmission
0,0.0,0.0,0.0,0.0,0.950,1.0,0.507319
1,0.0,0.0,0.0,0.0,0.900,1.0,0.507227
2,0.0,0.0,0.0,0.0,0.875,1.0,0.507229
3,0.0,0.0,0.0,0.0,0.675,1.0,0.507283
4,0.0,0.0,0.0,0.0,0.650,1.0,0.507230
...,...,...,...,...,...,...,...
77755,1.0,1.0,1.0,1.0,0.075,0.0,0.491165
77756,1.0,1.0,1.0,1.0,0.050,0.0,0.490764
77757,1.0,1.0,1.0,1.0,0.050,0.0,0.490698
77758,1.0,1.0,1.0,1.0,0.025,0.0,0.492394


In [45]:
testing_set.iloc[0][0:6]

Fill Factor    0.00
Pitch          0.00
Duty Cycle     0.00
Theta          0.00
Lambda         0.95
Mode           1.00
Name: 0, dtype: float64

In [53]:
x = torch.tensor(testing_set.iloc[1][0:6], dtype=torch.float32)
x

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 1.0000])

In [40]:
denominator = (4.378690e-01) - (-4.506890e-01)
denominator

0.888558

In [54]:
value = (testing_set.iloc[1]["Transmission"]*denominator) + (-4.506890e-01)
value

1.1300000000047383e-05

In [51]:
GratingCouplerNet(x)

tensor([0.5306], grad_fn=<AddBackward0>)

In [55]:
value = (GratingCouplerNet(x)*denominator) + (-4.506890e-01)
value

tensor([0.0193], grad_fn=<AddBackward0>)

In [56]:
loss_function = torch.nn.MSELoss()

In [59]:
loss_function(GratingCouplerNet(x), torch.tensor(testing_set.iloc[1]["Transmission"], dtype=torch.float32))

/home/jonathanlevine/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.0005, grad_fn=<MseLossBackward>)

In [61]:
(GratingCouplerNet(x) - torch.tensor(testing_set.iloc[1]["Transmission"]))**2

tensor([0.0005], grad_fn=<PowBackward0>)

In [64]:
abs((GratingCouplerNet(x) - torch.tensor(testing_set.iloc[1]["Transmission"])))/(torch.tensor(testing_set.iloc[1]["Transmission"]))

tensor([0.0427], grad_fn=<DivBackward0>)

In [65]:
training_set = pandas.read_csv('DATA_FILES/training_set_normalized_2.csv')
testing_set = pandas.read_csv('DATA_FILES/testing_set_normalized_2.csv')

In [66]:
testing_set

,Fill Factor,Pitch,Duty Cycle,Theta,Lambda,Mode,Transmission
0,0.0,0.0,0.0,0.0,1.000,1,0.000097
1,0.0,0.0,0.0,0.0,0.975,1,0.000100
2,0.0,0.0,0.0,0.0,0.975,1,0.000105
3,0.0,0.0,0.0,0.0,0.950,1,0.000094
4,0.0,0.0,0.0,0.0,0.925,1,0.000086
...,...,...,...,...,...,...,...
77755,1.0,1.0,1.0,1.0,0.075,0,-0.013321
77756,1.0,1.0,1.0,1.0,0.050,0,-0.014841
77757,1.0,1.0,1.0,1.0,0.050,0,-0.014831
77758,1.0,1.0,1.0,1.0,0.025,0,-0.014691


In [94]:
x = torch.tensor(testing_set.iloc[0][0:6], dtype=torch.float32)
x

tensor([0., 0., 0., 0., 1., 1.])

In [95]:
GratingCouplerNet(x)

tensor([-4.9092], grad_fn=<AddBackward0>)

In [82]:
np.log10(testing_set.iloc[1]["Transmission"])

-4.001304841688344

In [97]:
((-4.001) - torch.tensor(np.log10(testing_set.iloc[0]["Transmission"])))**2

tensor(0.0002, dtype=torch.float64)

In [85]:
np.log10(testing_set.iloc[0]["Transmission"])

-4.014124642691606